# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [37]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
import pickle

import spacy
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import ComplementNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, make_scorer, hamming_loss

In [2]:
# load data from database
conn = sqlite3.connect('../data/DisasterResponse.db')
df = pd.read_sql_query('SELECT * FROM message', conn, index_col='id')
X = df.message
Y = df.loc[:, 'related':]

In [3]:
X.head()

id
2     Weather update - a cold front from Cuba that c...
7               Is the Hurricane over or is it not over
8                       Looking for someone but no name
9     UN reports Leogane 80-90 destroyed. Only Hospi...
12    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """ Return lowercase lemmatized tokens of the input text. 
        Remove punctuation, spacy.stop_words, and pronouns.
    
    Args:
        text (str): input text
    Returns:
        tokens ([str]): list of lower case lemmatized tokens
    """
    
    doc = nlp(text)
    tokens = [tok.lemma_.lower() for tok in doc
              if (not tok.is_stop) and (not tok.is_punct) and (tok.lemma_ != '-PRON-')]
    return tokens

In [6]:
# test tokenize()
text = X.iloc[3]
print(text, '\n', tokenize(text))

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately. 
 ['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st.', 'croix', 'function', 'need', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Simple pipeline to classify messages with Count Vectorizer --> Multinomial Naive Bayes
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(ComplementNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# Unstratified split of data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [11]:
# train pipeline
%time pipeline.fit(X_train, Y_train)

Wall time: 3min 51s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000001BBFACB21F8>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=ComplementNB(alpha=1.0,
          

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
%time preds = pd.DataFrame(pipeline.predict(X_test), columns=Y_test.columns.copy())

Wall time: 1min 17s


In [13]:
report = classification_report(Y_test, preds, target_names=Y_test.columns.copy())

C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alexander\Anaconda3\lib\site-packages\skle

In [14]:
print(report)

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4995
               request       0.64      0.53      0.58      1112
                 offer       0.00      0.00      0.00        28
           aid_related       0.67      0.74      0.71      2719
          medical_help       0.39      0.14      0.21       555
      medical_products       0.33      0.11      0.16       342
     search_and_rescue       0.05      0.01      0.02       179
              security       0.00      0.00      0.00       123
              military       0.28      0.12      0.16       205
           child_alone       0.00      0.00      0.00         0
                 water       0.38      0.13      0.19       435
                  food       0.60      0.35      0.44       733
               shelter       0.43      0.20      0.27       571
              clothing       0.28      0.18      0.22       104
                 money       0.10      

In [17]:
print(f'Hamming Loss: {hamming_loss(Y_test, preds)}')

Hamming Loss: 0.06614625781037761


Base ComplementNB performs rather abysmally. This is most likely due to highly imbalanced classes in each category with class of interest being a severily imbalanced minority class. It is unlikely that pure model tuning will help.

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001BBFACB21F8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=ComplementNB(alpha=1.0, class_prior=None,
                                                fit_prior=True, norm=False),
                         n_jobs=None))],
 'verbose': False,
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='co

In [25]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__estimator__alpha': [1, 1e-1, 1e-2]
}

scorer = make_scorer(hamming_loss, greater_is_better=False)

cv = GridSearchCV(pipeline, parameters, cv=5, scoring=scorer)

In [26]:
%time cv.fit(X_train, Y_train)

Wall time: 11h 23min 32s


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
%time preds_cv = pd.DataFrame(cv.predict(X_test), columns=Y.columns.copy())

Wall time: 1min 15s


In [30]:
report_cv = classification_report(Y_test, preds_cv, target_names=Y_test.columns.copy())
print(report_cv)

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      4995
               request       0.76      0.46      0.57      1112
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.68      0.70      2719
          medical_help       0.42      0.03      0.05       555
      medical_products       0.61      0.03      0.06       342
     search_and_rescue       0.20      0.01      0.01       179
              security       0.00      0.00      0.00       123
              military       0.20      0.01      0.02       205
           child_alone       0.00      0.00      0.00         0
                 water       0.70      0.08      0.14       435
                  food       0.78      0.28      0.41       733
               shelter       0.78      0.12      0.20       571
              clothing       0.50      0.06      0.10       104
                 money       0.17      

In [31]:
print(f'Hamming Loss after CV: {hamming_loss(Y_test, preds_cv)}')

Hamming Loss after CV: 0.06001256452051073


Both cross-validation and test set splits do not account for class imbalance and are not stratified in the approach above, leading to poor model training.

Additionally, Hamming Loss scorer maybe too soft, leading to higher precision but lower recall. Since classification of messages is likely to be used to provide relief during natural disasters, it is more important to alert an appropriate agency of potential need and not miss any categories even if we create a false alert, rather than not place a message in a category and not alerting of a need when there is one. Thus, **recall is more important than precision in our classification**.

In [32]:
cv.best_score_

-0.05919793966151582

In [33]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000001BBFACB21F8>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l1', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=ComplementNB(alpha=0.1,
         

In [34]:
cv.best_params_

{'clf__estimator__alpha': 0.1,
 'tfidf__norm': 'l1',
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [40]:
# TODO:
# - Stratified train / test split
# - Stratified k-fold
# - Improved sampling (SMOTE, etc.) to address class imbalance -- use imblearn on per category basis

### 9. Export your model as a pickle file

In [41]:
with open('../models/model_01.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.